<a href="https://colab.research.google.com/github/bryanbayup/Machine-Learning/blob/main/Petpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import pandas as pd

df = pd.read_csv('dog_cat.csv')
print(df.head())

   Nama Penyakit Nama Hewan                                 Gejala  \
0  Panleukopenia     Kucing        Demam, muntah, diare, dehidrasi   
1         Rabies     Anjing      Perubahan perilaku, agresi, demam   
2  Leptospirosis     Anjing  Demam, muntah, gagal ginjal, kelesuan   
3     Parvovirus     Anjing          Muntah, diare berdarah, lemas   
4      Distemper     Anjing     Batuk, pilek, demam tinggi, kejang   

                                  Penanganan Pertama  
0  Isolasi kucing yang sakit, beri air minum yang...  
1  Pastikan untuk tidak terkena gigitan, isolasi ...  
2  Pastikan anjing tidak dehidrasi dengan memberi...  
3  Jaga anjing tetap hangat, beri cairan rehidras...  
4  Isolasi anjing, berikan kompres hangat untuk m...  


In [9]:
print(df.columns)
print(df.info())

Index(['Nama Penyakit', 'Nama Hewan', 'Gejala', 'Penanganan Pertama'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Nama Penyakit       500 non-null    object
 1   Nama Hewan          500 non-null    object
 2   Gejala              500 non-null    object
 3   Penanganan Pertama  500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB
None


In [10]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [11]:
df['Gejala'] = df['Gejala'].apply(clean_text)
df['Penanganan Pertama'] = df['Penanganan Pertama'].apply(clean_text)

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Menggabungkan semua teks untuk fit tokenizer
all_text = pd.concat([df['Gejala'], df['Penanganan Pertama']], axis=0)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

In [41]:
# Menambahkan token <start> dan <end>
tokenizer.word_index['<start>'] = len(tokenizer.word_index) + 1
tokenizer.word_index['<end>'] = len(tokenizer.word_index) + 1

In [42]:
start_token = tokenizer.word_index['<start>']
end_token = tokenizer.word_index['<end>']

vocab_size = len(tokenizer.word_index) + 1  # Tambahkan 1 untuk padding

In [57]:
# Konversi teks ke sequences
gejala_seq = tokenizer.texts_to_sequences(df['Gejala'])
penanganan_seq = tokenizer.texts_to_sequences(df['Penanganan Pertama'])

# Menambahkan token start dan end pada penanganan_seq
penanganan_seq = [[start_token] + seq + [end_token] for seq in penanganan_seq]

In [58]:
# Menghitung ulang max_length berdasarkan sequences yang telah diperbarui
max_length_gejala = max([len(seq) for seq in gejala_seq])
max_length_penanganan = max([len(seq) for seq in penanganan_seq])

# Menentukan max_length baru
max_length = max(max_length_gejala, max_length_penanganan)

In [70]:
# Melakukan padding pada sequences hingga max_length
gejala_padded = pad_sequences(gejala_seq, maxlen=max_length, padding='post')
penanganan_padded = pad_sequences(penanganan_seq, maxlen=max_length, padding='post')

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    gejala_padded, penanganan_padded, test_size=0.2, random_state=42)

In [82]:
# Menyiapkan data untuk decoder
decoder_input_data = y_train[:, :-1]  # Semua kecuali token terakhir
decoder_target_data = y_train[:, 1:]  # Mulai dari token kedua

# Tambahkan dimensi ekstra pada decoder_target_data
import numpy as np
decoder_target_data = np.expand_dims(decoder_target_data, -1)

In [83]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

embedding_dim = 128
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_length,))
enc_emb = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length - 1,))
dec_emb_layer = Embedding(vocab_size, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model Training
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile Model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [84]:
print("Shape X_train:", X_train.shape)
print("Shape decoder_input_data:", decoder_input_data.shape)
print("Shape decoder_target_data:", decoder_target_data.shape)
print("max_length:", max_length)

Shape X_train: (400, 23)
Shape decoder_input_data: (400, 22)
Shape decoder_target_data: (400, 22, 1)
max_length: 23


In [85]:
batch_size = 64
epochs = 50

model.fit(
    [X_train, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 721ms/step - loss: 4.7171 - val_loss: 4.6777
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - loss: 4.6316 - val_loss: 4.1596
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step - loss: 4.1096 - val_loss: 4.0316
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 237ms/step - loss: 4.0016 - val_loss: 3.9643
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - loss: 3.9378 - val_loss: 3.8825
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - loss: 3.8840 - val_loss: 3.8327
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - loss: 3.8054 - val_loss: 3.7764
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 237ms/step - loss: 3.7900 - val_loss: 3.5852
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - loss: 3.6486 - val_loss: 3.6347
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 347ms/step - loss: 3.5608 - val_loss: 3.4109
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 299ms/step - loss: 3.4707 - val_loss: 3.3444
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - loss: 3.3201 - val_lo

In [86]:
model.save('chatbot_model.h5')

In [87]:
encoder_model = Model(encoder_inputs, encoder_states)

In [88]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [89]:
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [90]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [91]:
reverse_word_index = dict(
    (i, word) for word, i in tokenizer.word_index.items())

def decode_sequence(input_seq):
    # Encode input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with only the start token.
    target_seq = np.array([[start_token]])

    # Sampling loop
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_word_index.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence.split()) > max_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

            # Update target sequence
            target_seq = np.array([[sampled_token_index]])

            # Update states
            states_value = [h, c]

    return decoded_sentence

In [92]:
def prepare_input(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    return padded

In [94]:
while True:
    input_text = input("Anda: ")
    if input_text.lower() in ['exit', 'quit']:
        break
    input_seq = prepare_input(input_text)
    response = decode_sequence(input_seq)
    print("Chatbot:", response)

Anda: kucing saya sakit


NameError: name 'remove_stopwords' is not defined